# Feature Extraction: Standard and Melody-Based Audio Features

In this notebook, both standard and melody-related audio features are extracted. Standard audio features are obtained using the `pyAudioAnalysis` package, and a comprehensive list of these features can be found [here](https://github.com/tyiannak/pyAudioAnalysis/wiki/3.-Feature-Extraction).

Melody features are extracted using the `Melodia` plug-in developed by Justin Salamon. More details about `Melodia` can be found on the [website](https://www.upf.edu/web/mtg/melodia). The extraction of the features is performed based on the paper by Salamon et al., which is available [here](https://www.justinsalamon.com/uploads/4/3/9/4/4394963/salamonrochagomezicassp2012.pdf).

## Packages

In [1]:
# general
import os
import numpy as np
import pandas as pd
import re
from tqdm import tqdm

# librosa
import librosa
import librosa.display

# pyAudioAnalysis
from pyAudioAnalysis import ShortTermFeatures as stF
from pyAudioAnalysis import MidTermFeatures as mtF

# to load Melodia plug-in
import vamp

# to play the audio files
import IPython.display as ipd
from IPython.display import Audio

# plots
import seaborn as sns
import matplotlib.pyplot as plt

/home/gosia/miniconda3/envs/data_music/lib/python3.12/site-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


## Functions

### Standard features: pyAudioAnalysis and librosa 

In [2]:
def extract_standard_features(audio_file):
    
    data, sr = librosa.load(audio_file, sr=None)

    # librosa features
    tempo_librosa, _ = librosa.beat.beat_track(y=data.astype('float'), sr=sr, units="time")

    # pyAudioAnalysis
    win, step = 0.050, 0.050
    [f, fn] = stF.feature_extraction(data, sr, int(sr * win), 
                                    int(sr * step))
    mt, st, mt_n = mtF.mid_feature_extraction(data, sr, len(data), len(data), 
                                              0.05 * sr, 0.05 * sr)
    tempo_pyAA, _ = mtF.beat_extraction(stF.feature_extraction(data, sr, int(sr * win), int(sr * step))[0], win)

    df_mt = pd.DataFrame(mt.T)
    df_mt.columns = [mt_n[i]+'_pyAA' for i in range(len(mt_n))]
    df_mt['tempo_pyAA'] = tempo_pyAA


    df = {'sr': [sr],
          'samples': [len(data)],
          'duration_sec': [len(data)/sr],
          'tempo_librosa': [tempo_librosa]
          }

    df = pd.DataFrame.from_dict(df)
    df = pd.concat([df, df_mt], axis=1, join="inner")
    
    return df

### Melody features

In [3]:
def vibrato_stats(contour_hz):

    # transform to cents
    contour = 1200 * np.log2(contour_hz/55)

    # hop
    H = 128
    # sampling rate
    f_s = 44100

    df = pd.DataFrame({
            'vibrato_length': [0],
            'vibrato_coverage': [0],
            'vibrato_rate_hz': [np.nan],
            'vibrato_extend_cents': [np.nan],
            'vibrato_extend_hz': [np.nan]
    })

    if contour_hz.shape[0] < 125:
        return df

    # window size
    n_fft_contour = np.min([180, contour_hz.shape[0]-5])

    pitches, magnitudes = librosa.piptrack(y=contour_hz, sr=f_s/H, n_fft=n_fft_contour, hop_length=1, ref=np.mean, fmin=1, center=True)
    pitches[pitches==0] = np.nan
    # range of human vibrato
    vibrato_ind = ((1*(pitches >= 5) + 1*(pitches <= 8)) == 2)

    if np.sum(np.any(vibrato_ind, axis=0)) > 0:

        df = pd.DataFrame({
            'vibrato_length': [np.sum(np.any(vibrato_ind, axis=0))*H/f_s],
            'vibrato_coverage': [np.sum(np.any(vibrato_ind, axis=0))/vibrato_ind.shape[1]],
            'vibrato_rate_hz': [np.mean(pitches[vibrato_ind])],
            'vibrato_extend_cents': [1200 * np.log2(np.mean(magnitudes[vibrato_ind])/55)],
            'vibrato_extend_hz': [np.mean(magnitudes[vibrato_ind])]
        })

    return df


def contour_extract_features(contour_hz):

    # transform to cents
    contour = 1200 * np.log2(contour_hz/55)

    # hop
    H = 128
    # sampling rate
    f_s = 44100

    df = pd.DataFrame({
        'duration': [(contour).shape[0]*H/f_s],
        'pitch_mean_height_cents': [np.mean(contour)],
        'pitch_deviation_cents': [np.std(contour)],
        'pitch_range_cents': [np.max(contour) - np.min(contour)],
        'pitch_min_cents': [np.min(contour)],
        'pitch_max_cents': [np.max(contour)],
        'pitch_mean_height_hz': [np.mean(contour_hz)],
        'pitch_deviation_hz': [np.std(contour_hz)],
        'pitch_range_hz': [np.max(contour_hz) - np.min(contour_hz)],
        'pitch_min_hz': [np.min(contour_hz)],
        'pitch_max_hz': [np.max(contour_hz)]
    })

    df_vibrato = vibrato_stats(contour_hz)
    df = pd.concat([df, df_vibrato], axis=1, join="inner")
    
    return df


def song_contour_extract_melody_features(melody_hz):

    df = pd.DataFrame()

    melody_hz_tmp = np.hstack((melody_hz, np.nan))
    
    start_ind = np.where(~np.isnan(melody_hz_tmp))[0]

    interval_length = []

    while start_ind.shape[0] > 0:
        interval_length.append(start_ind[0])
        melody_hz_tmp = melody_hz_tmp[start_ind[0]:]
        end_ind = np.where(np.isnan(melody_hz_tmp))[0]

        contour_hz_tmp = melody_hz_tmp[:end_ind[0]]

        if contour_hz_tmp.shape[0] > 50:
            df_tmp = contour_extract_features(contour_hz_tmp)
            df = pd.concat([df, df_tmp], ignore_index = True)

        melody_hz_tmp = melody_hz_tmp[(end_ind[0]+1):]
        start_ind = np.where(~np.isnan(melody_hz_tmp))[0]

    interval_length = interval_length[1:]
    
    return df, interval_length


def melody_features_from_df(df_all):

    df_tmp = pd.concat([df_all.mean(),
                        df_all.std()])
    df_tmp = pd.DataFrame(df_tmp).T

    df_tmp.columns = [df_all.columns[i] +'_mean' for i in range(len(df_all.columns))] + \
        [df_all.columns[i] +'_std' for i in range(len(df_all.columns))] 
    
    df_global = pd.DataFrame({
        'global_highest_pitch_cents': [np.max(df_all['pitch_max_cents'])],
        'global_lowest_pitch_cents': [np.min(df_all['pitch_min_cents'])],
        'global_pitch_range_cents': [np.max(df_all['pitch_max_cents']) - np.min(df_all['pitch_min_cents'])],
        'global_vibrato_coverage': [df_all['vibrato_length'].sum()/df_all['duration'].sum()]
    })

    df = pd.concat([df_global, df_tmp], axis=1, join="inner")

    return df


def extract_melody_features(audio_file):

    data, sr = librosa.load(audio_file, sr=None)

    data_melodia_melody = vamp.collect(data, sr, "mtg-melodia:melodia", output='melody')
    hop, melody_hz = data_melodia_melody['vector']
    melody_hz[melody_hz<=0] = np.nan
    melody = 1200 * np.log2(melody_hz/55)

    df_all, interval_lengths = song_contour_extract_melody_features(melody_hz)

    df100 = melody_features_from_df(df_all)
    df100.columns = [df100.columns[i] + '_all' for i in range(len(df100.columns))]
    df100['interval_duration_mean_all'] = np.mean(interval_lengths)
    df100['interval_duration_std_all'] = np.std(interval_lengths)

    # observation with duration above q_1/3 (2/3 of observations chosen)
    toptwothird_duration = (df_all['duration'] >= np.array(df_all['duration'].sort_values())[int(np.floor(len(df_all)*1/3))])
    df_all_twothird = df_all.loc[toptwothird_duration,:]
    df_twothird = melody_features_from_df(df_all_twothird)
    df_twothird.columns = [df_twothird.columns[i] + '_twothird' for i in range(len(df_twothird.columns))]

    # observation with duration above q_2/3 (1/3 of observations chosen)
    toponethird_duration = (df_all['duration'] >= np.array(df_all['duration'].sort_values())[int(np.floor(len(df_all)*2/3))])
    df_all_onethird = df_all.loc[toponethird_duration,:]
    df_onethird = melody_features_from_df(df_all_onethird)
    df_onethird.columns = [df_onethird.columns[i] + '_onethird' for i in range(len(df_onethird.columns))]

    df = pd.concat([df100, df_twothird], axis=1, join="inner")
    df = pd.concat([df, df_onethird], axis=1, join="inner")

    return df

### Extract features for all files

In [4]:
def extract_features(file_paths):

    # standard features
    df_SF = pd.DataFrame()
    # melody features
    df_MF = pd.DataFrame()

    for file_path in tqdm(file_paths):
        df_SF_tmp = extract_standard_features(file_path)
        df_SF_tmp[['file_path']] = file_path
        df_SF_tmp[['song_id']] = re.findall('[0-9]+', file_path)[0]
        df_SF = pd.concat([df_SF, df_SF_tmp], ignore_index = True)

        df_MF_tmp = extract_melody_features(file_path)
        df_MF_tmp[['file_path']] = file_path
        df_MF_tmp[['song_id']] = re.findall('[0-9]+', file_path)[0]
        df_MF = pd.concat([df_MF, df_MF_tmp], ignore_index = True)

    return df_SF, df_MF

## List of files

In [5]:
audio = 'audio_path'
music_directory = os.listdir(audio)

In [6]:
file_paths = []
for file_name in music_directory:
    # Create the full file path using os.path.join()
    file_path = os.path.join(audio, file_name)
    
    # Add the file path to the list
    file_paths.append(file_path)
    
    
def extract_numeric_part(file_path):
    return [int(s) for s in os.path.basename(file_path).split('.') if s.isdigit()][0]

# Sort the file paths based on the numeric values in the file names
file_paths = sorted(file_paths, key=extract_numeric_part)
# Delete 'zone identifier':
file_paths = [file_path for file_path in file_paths if not 'Zone.Identifier' in file_path]

## Extract features and save

In [7]:
df_SF, df_MF = extract_features(file_paths)

100%|██████████| 903/903 [1:26:10<00:00,  5.73s/it]


In [8]:
df_SF.to_csv('data/audio/preprocessed_SF.csv')

In [9]:
df_MF.to_csv('data/audio/preprocessed_MF.csv')